In [1]:
import pickle
import numpy as np
from nilearn import plotting, datasets
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import stats
import os
import glob
import re

In [2]:
atlas_labels = datasets.fetch_atlas_aal()['labels']
# mpl.use("agg")

In [3]:
with open("FCs.pkl", "rb") as f:
    fcs:dict = pickle.load(f)

In [71]:
# 选择一个合适的二值化阈值
# 以全局效率为标准，绘制不同二值化阈值下全局效率统计学差异变化曲线
thresholds = range(35, 80, 2)
save_path = "graph_theory/cluster"
os.makedirs(save_path, exist_ok=True)
mw_stats = []
mw_p_values = []
for threshold in thresholds:
    if len(fcs[sub]["ses-preop"]) == 0:
        continue
    index1 = []
    index2 = []
    for sub in fcs:
        for run, fc in fcs[sub]["ses-preop"].items():
            # 创建图
            fcg = nx.Graph()
            fcg.add_nodes_from(atlas_labels)
            fcg.add_edges_from([(atlas_labels[i], atlas_labels[j]) for i in range(fc.shape[0]) for j in range(fc.shape[1]) if i != j and abs(fc[i][j]) >= threshold/100])
            if len(fcg.nodes) == 0:
                print(f"图为空：{sub}-ses-preop-{run}-{threshold}")
                # GE.append(GE[-1])
            else:
                index1.append(sum(nx.clustering(fcg).values()))
                # index1.append(nx.degree_assortativity_coefficient(fcg))
                if np.isnan(index1[-1]):
                    index1.pop()
        for run, fc in fcs[sub]["ses-postop"].items():
            # 创建图
            fcg = nx.Graph()
            fcg.add_nodes_from(atlas_labels)
            fcg.add_edges_from([(atlas_labels[i], atlas_labels[j]) for i in range(fc.shape[0]) for j in range(fc.shape[1]) if i != j and abs(fc[i][j]) >= threshold/100])
            if len(fcg.nodes) == 0:
                print(f"图为空：{sub}-ses-postop-{run}-{threshold}")
                # GE.append(GE[-1])
            else:
                index2.append(sum(nx.clustering(fcg).values()))
                # index2.append(nx.degree_assortativity_coefficient(fcg))
                if np.isnan(index2[-1]):
                    index2.pop()
    mw_stat, mw_p_value = stats.mannwhitneyu(index1, index2)
    mw_stats.append(mw_stat)
    mw_p_values.append(mw_p_value)
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
fig.patch.set_color("white")
ax[0].set_title("mannwhitney u-stat")
ax[0].plot(thresholds, mw_stats)
ax[1].set_title("mannwhitney u p-value")
ax[1].plot(thresholds, mw_p_values)
fig.savefig(f"{save_path}/mw_stat.png", format="png")
plt.cla()
plt.clf()
plt.close("all")

In [52]:
threshold = 45

In [ ]:
# 计算图论参数并保存
save_path = "graph_theory/pkl"
os.makedirs(save_path, exist_ok=True)
for sub in fcs:
    # if os.path.exists(f"{save_path}/{sub}.pkl"):
    #     continue
    fc_graph_theory= {}
    for ses in fcs[sub]:
        fc_graph_theory[ses] = {}
        for run, fc in fcs[sub][ses].items():

            # %%
            # 创建图
            fcg = nx.Graph()
            fcg.add_edges_from([(atlas_labels[i], atlas_labels[j]) for i in range(fc.shape[0]) for j in range(fc.shape[1]) if i != j and fc[i][j] >= threshold/100])
            fcg.add_nodes_from(atlas_labels)
            if len(fcg.nodes) == 0:
                print(f"图为空：{sub}-{ses}-{run}")
                continue
            fc_graph_theory[ses][run] = {}
            this_run = fc_graph_theory[ses][run]
            this_run["graph"] = fcg

            # %%
            # # degree（度）
            # this_run["degree"] = fcg.degree

            # k-core（k度核）
            this_run["main_core"] = nx.k_core(fcg)

            # # %%
            # # 中心性
            # # degree centrality（度中心性）
            # this_run["degree_centrality"] = nx.degree_centrality(fcg)

            # # closeness centrality（接近中心性）
            # this_run["closeness_centrality"] = nx.closeness_centrality(fcg)

            # # betweenness centrality（中介中心性）
            # this_run["betweenness_centrality"] = nx.betweenness_centrality(fcg)

            # try:
            #     # Eigenvector Centrality（特征向量中心性）
            #     this_run["eigenvector_centrality"] = nx.eigenvector_centrality(fcg)
            # except Exception as e:
            #     print(f"{sub}-{ses}-{run}: ", e)
            #     fc_graph_theory[ses].pop(run)
            #     continue

            # %%
            # 聚类性质
            # # numbers of triangles（三角形数）
            # this_run["triangles"] = nx.triangles(fcg)

            # clustering efficiency（聚类系数）
            this_run["clustering"] = nx.clustering(fcg)

            # transitivity（传递性）
            this_run["transitivity"] = nx.transitivity(fcg)

            # %%
            # 关联性
            # Network Assortativity（网络关联性）
            this_run["degree_assortativity_coefficient"] = nx.degree_assortativity_coefficient(fcg)

            # average neighbor degree（平均邻居度）
            this_run["average_neighbor_degree"] = nx.average_neighbor_degree(fcg)

            # average degree connectivity（平均度连接）
            this_run["average_degree_connectivity"] = nx.average_degree_connectivity(fcg)

            # %%
            # 效率性质
            # global efficiency（全局效率）
            this_run["global_efficiency"] = nx.global_efficiency(fcg)

            # local efficiency（局部效率）
            this_run["local_efficiency"] = nx.local_efficiency(fcg)

            # ratio of local to global efficiency（局部全局效率比）
            this_run["ratio"] = this_run["local_efficiency"]/this_run["global_efficiency"]

#             # %%
#             try:
#                 # rich club
#                 rich_club = nx.rich_club_coefficient(fcg)
#             except Exception as e:
#                 print(e)

#             try:
#                 # characteristic path length/average shortest path length（特征路径长度/平均最短路径长度）
#                 this_run["char_path_len"] = nx.average_shortest_path_length(fcg)

# #                 # small-world coefficient（小世界系数）
# #                 this_run["small_world"] = nx.sigma(fcg, 30)
#             except nx.NetworkXError as e:
#                 print(f"{e} : {sub}-{ses}-{run}")

            # %%
            print(f"计算完成：{sub}-{ses}-{run}")
            with open(f"{save_path}/{sub}.pkl", "wb") as f:
                pickle.dump(fc_graph_theory, f)

计算完成：sub-292-ses-preop-run-01
计算完成：sub-292-ses-preop-run-02
计算完成：sub-292-ses-postop-run-02
计算完成：sub-292-ses-postop-run-03
计算完成：sub-292-ses-postop-run-04
计算完成：sub-292-ses-postop-run-05
计算完成：sub-294-ses-preop-run-01
计算完成：sub-294-ses-postop-run-01
计算完成：sub-294-ses-postop-run-02
计算完成：sub-302-ses-preop-run-01
计算完成：sub-302-ses-postop-run-01
计算完成：sub-302-ses-postop-run-02
计算完成：sub-303-ses-preop-run-01
计算完成：sub-303-ses-preop-run-02
计算完成：sub-303-ses-preop-run-03
计算完成：sub-303-ses-preop-run-04
计算完成：sub-303-ses-preop-run-05
计算完成：sub-303-ses-postop-run-01
计算完成：sub-303-ses-postop-run-02
计算完成：sub-303-ses-postop-run-03
计算完成：sub-303-ses-postop-run-04
计算完成：sub-303-ses-postop-run-05
计算完成：sub-303-ses-postop-run-06
计算完成：sub-307-ses-preop-run-01
计算完成：sub-307-ses-preop-run-02
计算完成：sub-307-ses-preop-run-03
计算完成：sub-307-ses-preop-run-04
计算完成：sub-307-ses-preop-run-05
计算完成：sub-307-ses-postop-run-01
计算完成：sub-307-ses-postop-run-02
计算完成：sub-307-ses-postop-run-03
计算完成：sub-307-ses-postop-run-04
计算完成：sub-307-ses-posto

In [ ]:
graphs = {}
for file in glob.glob("graph_theory/pkl/static/*"):
    with open(file, "rb") as f:
        graph = pickle.load(f)
    if len(graph["ses-preop"]) > 0:
        graphs["sub-"+re.search(r"\d+", file)[0]] = graph
with open("graph_theory/graphs.pkl", "wb") as f:
    pickle.dump(graphs, f)